In [7]:
import pandas as pd
import numpy as np 
df = pd.read_csv('../../world_gdp.csv')
df.head()
# governance.dtypes
# governance.describe(include="all")

,Year,Country Name,Country Code,Current GDP ($),GDP growth (annual %),GDP per capita ($),Gini index,%individuals using internet,Interest payments (% of expense),Interest payments (% of revenue),Land area (sq.km),Life expectancy at birth,Military expenditure (% of GDP)
0,2016,Afghanistan,AFG,18116562464.9088,2.26031420130452,512.012778129357,..,..,0.108067197970998,0.0915435039044285,652860,63.763,0.956772140958855
1,2017,Afghanistan,AFG,18753469630.2586,2.64700320195786,516.679862210543,..,..,0.134207231842939,0.109378878694535,652860,64.13,0.945226799939516
2,2020,Afghanistan,AFG,20116137325.8206,-2.35110067344623,516.747870807558,..,..,..,..,652860,..,1.36968441109766
3,2018,Afghanistan,AFG,18053228578.8878,1.18922812838034,485.668418729808,..,..,..,..,652860,64.486,1.00674629248273
4,2019,Afghanistan,AFG,18799450742.7823,3.911603419423,494.179349886029,..,..,..,..,652860,64.833,1.11823060279695


In [8]:
df=df.replace({'..':np.NaN})#replace all cells with .. with None so that python can evaluate it as missing value
df.head()

,Year,Country Name,Country Code,Current GDP ($),GDP growth (annual %),GDP per capita ($),Gini index,%individuals using internet,Interest payments (% of expense),Interest payments (% of revenue),Land area (sq.km),Life expectancy at birth,Military expenditure (% of GDP)
0,2016,Afghanistan,AFG,18116562464.9088,2.26031420130452,512.012778129357,NaN,NaN,0.108067197970998,0.0915435039044285,652860,63.763,0.956772140958855
1,2017,Afghanistan,AFG,18753469630.2586,2.64700320195786,516.679862210543,NaN,NaN,0.134207231842939,0.109378878694535,652860,64.13,0.945226799939516
2,2020,Afghanistan,AFG,20116137325.8206,-2.35110067344623,516.747870807558,NaN,NaN,NaN,NaN,652860,NaN,1.36968441109766
3,2018,Afghanistan,AFG,18053228578.8878,1.18922812838034,485.668418729808,NaN,NaN,NaN,NaN,652860,64.486,1.00674629248273
4,2019,Afghanistan,AFG,18799450742.7823,3.911603419423,494.179349886029,NaN,NaN,NaN,NaN,652860,64.833,1.11823060279695


In [9]:
df.dtypes

Year                                 int64
Country Name                        object
Country Code                        object
Current GDP ($)                     object
GDP growth (annual %)               object
GDP per capita ($)                  object
Gini index                          object
%individuals using internet         object
Interest payments (% of expense)    object
Interest payments (% of revenue)    object
Land area (sq.km)                   object
Life expectancy at birth            object
Military expenditure (% of GDP)     object
dtype: object

# changes data types accordingly

In [10]:
object_list = df.select_dtypes(include=['object']).columns.to_list()[2:]
# print(object_list)
df[object_list]=df[object_list].astype(float)

df.dtypes

Year                                  int64
Country Name                         object
Country Code                         object
Current GDP ($)                     float64
GDP growth (annual %)               float64
GDP per capita ($)                  float64
Gini index                          float64
%individuals using internet         float64
Interest payments (% of expense)    float64
Interest payments (% of revenue)    float64
Land area (sq.km)                   float64
Life expectancy at birth            float64
Military expenditure (% of GDP)     float64
dtype: object

In [12]:
dic = {2016:0,2017:1,2018:2,2019:3,2020:4}
df['Age']=df['Year'].map(dic)
df.sort_values(['Country Name', 'Age'],inplace=True)

In [15]:
def fill_missing_values(group):
    value_columns = group.select_dtypes(include='number').columns

    for value_col in value_columns:
        values = group[value_col]
        known_values_mask = values.notnull()
        if known_values_mask.sum() > 0:  # If there is at least one known value
            if known_values_mask.sum() > 1:  # If there are more than one known values
                sorted_group = group.sort_values('Age')
                x_known = group.loc[known_values_mask, 'Age']
                y_known = group.loc[known_values_mask, value_col]
                a = (y_known.iloc[-1] - y_known.iloc[0]) / (x_known.iloc[-1] - x_known.iloc[0])
                b = y_known.iloc[0] - (a * x_known.iloc[0])

                missing_mask = values.isnull()
                x_missing = group.loc[missing_mask, 'Age']
                group.loc[missing_mask, value_col] = a * x_missing + b
            else:  # If there is only one known value
                known_value = values[known_values_mask].iloc[0]
                group[value_col].fillna(known_value, inplace=True)


    return group
final = df.reset_index().groupby('Country Name',group_keys=True).apply(fill_missing_values).reset_index(drop=True).drop('index',axis=1)
final

,Year,Country Name,Country Code,Current GDP ($),GDP growth (annual %),GDP per capita ($),Gini index,%individuals using internet,Interest payments (% of expense),Interest payments (% of revenue),Land area (sq.km),Life expectancy at birth,Military expenditure (% of GDP),Age
0,2016,Afghanistan,AFG,1.811656e+10,2.260314,512.012778,NaN,NaN,0.108067,0.091544,652860.0,63.763000,0.956772,0
1,2017,Afghanistan,AFG,1.875347e+10,2.647003,516.679862,NaN,NaN,0.134207,0.109379,652860.0,64.130000,0.945227,1
2,2018,Afghanistan,AFG,1.805323e+10,1.189228,485.668419,NaN,NaN,0.160347,0.127214,652860.0,64.486000,1.006746,2
3,2019,Afghanistan,AFG,1.879945e+10,3.911603,494.179350,NaN,NaN,0.186487,0.145050,652860.0,64.833000,1.118231,3
4,2020,Afghanistan,AFG,2.011614e+10,-2.351101,516.747871,NaN,NaN,0.212627,0.162885,652860.0,65.189667,1.369684,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1325,2016,Zimbabwe,ZWE,2.054868e+10,0.755869,1464.588957,41.3,23.119989,2.747953,4.033116,386850.0,60.294000,1.742494,0
1326,2017,Zimbabwe,ZWE,1.758489e+10,4.709492,1235.189032,44.3,24.400000,3.669340,6.024985,386850.0,60.812000,1.544948,1
1327,2018,Zimbabwe,ZWE,1.811554e+10,4.824211,1254.642265,47.3,25.000000,9.962580,13.473282,386850.0,61.195000,1.222795,2
1328,2019,Zimbabwe,ZWE,1.928429e+10,-6.144236,1316.740657,50.3,25.100000,13.569893,18.193365,386850.0,61.490000,0.698601,3


In [17]:
final.fillna(value=0,inplace=True)

In [18]:
from ydata_profiling import  ProfileReport
prof = ProfileReport(final,title='eda',explorative=True,dark_mode=True)
prof.to_file('output_file_cleaned.html')
# prof

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
final.to_csv('world_gdp_cleaned.csv')